<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/multitaxa/multitaxa_train_tf2_rcnns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train Tensorflow Faster-RCNN models to detect snakes & lizards (Squamata), beetles (Coleoptera), frogs (Anura), and carnivores (Carnivora)  from EOL images
---   
*Last Updated 3 March 2023*  
-Runs in Python 3 with Tensorflow 2.0-     

Use EOL user generated cropping coordinates to train Faster-RCNN and Faster-RCNN Inception Object Detection Models implemented in Tensorflow to detect animals from EOL images. Training data consists of the user-determined best square thumbnail crop of an image, so model outputs will also be a square around objects of interest.

Datasets were downloaded to Google Drive in [multitaxa_preprocessing.ipynb](https://github.com/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/multitaxa/multitaxa_preprocessing.ipynb).

***Models were trained in Python 2 and TF 1 in April 2020: Faster RCNN ResNet 50 trained for 12 hours to 200,000 steps and Faster RCNN Inception v2 for 18 hours to 200,000 steps.*** 

Notes:   
* Run code blocks by pressing play button in brackets on left
* Before you you start: change the runtime to "GPU" with "High RAM"
* Change parameters using form fields on right (find details at corresponding lines of code by searching '#@param')

References:     
* [Official Tensorflow Object Detection API Instructions](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html) 
* [Medium Blog on training using Tensorflow Object Detection API in Colab](https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b)

## Installs & Imports
---

In [ ]:
#@title Choose where to save results, set up directory structure & Build Tensorflow Object Detection API (if in Colab runtime, use defaults)
import os

# Choose where to save results
save = "in my Google Drive" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]

# Mount google drive to export image cropping coordinate file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type in the path to your project wd in form field on right
basewd = "/content/drive/MyDrive/train" #@param ["/content/drive/MyDrive/train"] {allow-input: true}

# Type in the folder that you want to contain TF2 files
folder = "tf2" #@param ["tf2"] {allow-input: true}
cwd = basewd + '/' + folder

# Enter taxon of interest in form field
taxon = "Multitaxa" #@param ["Multitaxa"] {allow-input: true}

# Make folders if they don't already exist
if not os.path.exists(cwd):
    os.makedirs(cwd)
%cd $cwd
if not os.path.exists("tf_models"):
    os.mkdir("tf_models")
    os.mkdir("results")
    %cd tf_models
    # Clone the Tensorflow Model Garden
    !git clone --depth 1 https://github.com/tensorflow/models
# Build Object Detection API
%cd $cwd
!cd tf_models/models/research/ && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

print("\nWorking directory set to:")
%cd $cwd

In [ ]:
# For object detection
import tensorflow as tf 
import tensorflow_hub as hub
import sys
sys.path.append("tf_models/models/research/")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
!pip install absl-py # TF1->TF2 compatibility hacks
!pip install lvis

# For downloading and displaying images
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
!pip install opencv-python-headless==4.1.2.30
import cv2
import tempfile
import urllib
from urllib.request import urlretrieve
from six.moves.urllib.request import urlopen
from six import BytesIO
from collections import defaultdict
from io import StringIO
from IPython.display import display

# For drawing onto images
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring inference time
import time

# For working with data
import numpy as np
import pandas as pd
import os
import pathlib
import csv
import six.moves.urllib as urllib
import tarfile
import zipfile
import shutil
import glob

# Print Tensorflow version
print('\nTensorflow Version: %s' % tf.__version__)

# Check available GPU devices
print('The following GPU devices are available: %s' % tf.config.experimental.get_device_details(tf.config.list_physical_devices('GPU')[0]))

## Model preparation (only run once)
---
These blocks download and set-up files needed for training object detectors. After running once, you can train and re-train as many times as you'd like. Pre-trained models are downloaded from the [Tensorflow Object Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

In [ ]:
#@title Download pre-trained Faster RCNN Resnet & Inception models from Tensorflow Object Detection Model Zoo
# Code modified from https://github.com/RomRoc/objdet_train_tensorflow_colab/blob/master/objdet_custom_tf_colab.ipynb

# Faster RCNN
# Define parameters
MODEL = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
DEST_DIR = 'tf_models/train_demo/rcnn/pretrained_model'
# Set up directory structure
if not (os.path.exists('tf_models/train_demo')):
    print("Setting up directory structure for ", MODEL)
    !mkdir -p tf_models/train_demo/rcnn
    %cd tf_models/train_demo/rcnn
    !mkdir {pretrained_model,finetuned_model,trained}
    %cd $cwd
# Download the model
if not (os.path.exists(MODEL_FILE)):
    print("\nDownloading model from {} to {}".format(DOWNLOAD_BASE, DEST_DIR))
    urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    tar = tarfile.open(MODEL_FILE)
    tar.extractall()
    tar.close()
    os.remove(MODEL_FILE)
    # Clean up directories from unzipping model
    if (os.path.exists(DEST_DIR)):
        shutil.rmtree(DEST_DIR)
    os.rename(MODEL, DEST_DIR)

# Faster RCNN Inception v2
# Define parameters
MODEL = 'faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
DEST_DIR = 'tf_models/train_demo/rcnn_i/pretrained_model'
# Set up directory structure
if not (os.path.exists('tf_models/train_demo')):
    print("Setting up directory structure for ", MODEL)
    !mkdir -p tf_models/train_demo/rcnn_i
    %cd tf_models/train_demo/rcnn_i
    !mkdir {pretrained_model,finetuned_model,trained}
    %cd $cwd
# Download the model
if not (os.path.exists(MODEL_FILE)):
    print("\nDownloading model from {} to {}".format(DOWNLOAD_BASE, DEST_DIR))
    urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    tar = tarfile.open(MODEL_FILE)
    tar.extractall()
    tar.close()
    os.remove(MODEL_FILE)
    # Clean up directories from unzipping model
    if (os.path.exists(DEST_DIR)):
        shutil.rmtree(DEST_DIR)
    os.rename(MODEL, DEST_DIR)

In [ ]:
#@title Convert test data to tf.record format
# Modified from https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

# Download multitaxa_generate_tfrecord.py
print("Downloading multitaxa_generate_tfrecord.py...\n")
!pip3 install --upgrade gdown
!gdown --id 1_pRlENeAvGV-h_c-_rl2d0Y1QxMJ0TAS

# Update file paths in form fields
csv_input = "/content/drive/MyDrive/train/tf2/pre-processing/Multitaxa_crops_test_notaug_oob_rem_fin.csv" #@param ["/content/drive/MyDrive/train/tf2/pre-processing/Multitaxa_crops_test_notaug_oob_rem_fin.csv"] {allow-input: true}
output_path = "/content/drive/MyDrive/train/tf2/test_images/tf.record" #@param ["/content/drive/MyDrive/train/tf2/test_images/tf.record"] {allow-input: true}
test_image_dir = "/content/drive/MyDrive/train/tf2/test_images" #@param ["/content/drive/MyDrive/train/tf2/test_images"] {allow-input: true}

# Generate tf.record for test images
!python multitaxa_generate_tfrecord.py --output_path=$output_path  --csv_input=$csv_input  --image_dir=$test_image_dir 

In [ ]:
#@title Convert train data to tf.record format
# Modified from https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

# Update file paths in form fields
csv_input = "/content/drive/MyDrive/train/tf2/pre-processing/Multitaxa_crops_train_aug_oob_rem_fin.csv" #@param ["/content/drive/MyDrive/train/tf2/pre-processing/Multitaxa_crops_train_aug_oob_rem_fin.csv"] {allow-input: true}
output_path = "/content/drive/MyDrive/train/tf2/images/tf.record" #@param ["/content/drive/MyDrive/train/tf2/images/tf.record"] {allow-input: true}
train_image_dir = "/content/drive/MyDrive/train/tf2/images" #@param ["/content/drive/MyDrive/train/tf2/images"] {allow-input: true}

# Generate tf.record for train images
!python multitaxa_generate_tfrecord.py --output_path=$output_path  --csv_input=$csv_input  --image_dir=$train_image_dir 

In [ ]:
#@title Make labelmap.pbtxt for taxa of interest

%%writefile labelmap.pbtxt
item {
  id: 1
  name: 'Squamata'
}

item {
  id: 2
  name: 'Coleoptera'
}

item {
  id: 3
  name: 'Anura'
}

item {
  id: 4
  name: 'Carnivora'
}

### Modify model config files for training Faster-RCNN Resnet and Faster-RCNN Inception with your dataset

If you have errors with training, check the pipline_config_path and model_dir in the config files for R-FCN or Faster-RCNN model

In [ ]:
# Adjust model config file based on training/testing datasets
# Modified from https://stackoverflow.com/a/63645324
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2
%cd $cwd

# Adjust parameters using form fields on right
filter = taxon # defined in first code block
config_basepath = "tf_models/train_demo/" #@param ["tf_models/train_demo/"] {allow-input: true}
label_map = 'labelmap.pbtxt'
train_tfrecord_path = "/content/drive/MyDrive/train/tf2/pre-processing/images/tf.record" #@param ["/content/drive/MyDrive/train/tf2/images/tf.record"] {allow-input: true}
test_tfrecord_path = "/content/drive/MyDrive/train/tf2/pre-processing/test_images/tf.record" #@param ["/content/drive/MyDrive/train/tf2/test_images/tf.record"] {allow-input: true}
ft_ckpt_basepath = "/content/drive/MyDrive/train/tf2/tf_models/train_demo/" #@param ["/content/drive/MyDrive/train/tf2/tf_models/train_demo/"] {allow-input: true}
ft_ckpt_type = "detection" #@param ["detection", "classification"]
num_classes = 4 #@param ["4"] {type:"raw", allow-input: true}
batch_size = 1 #@param ["1", "4", "8", "16", "32", "64", "128"] {type:"raw"}

# Define pipeline for modifying model config files
def read_config(model_config):
    if 'rcnn/' in model_config:
        model_ckpt = 'rcnn/pretrained_model/checkpoint/ckpt-0'
    elif 'rcnn_i/' in model_config:
        model_ckpt = 'rcnn_i/pretrained_model/checkpoint/ckpt-0'
    config_fpath = config_basepath + model_config
    pipeline = pipeline_pb2.TrainEvalPipelineConfig()                                                                                                                                                                                                          
    with tf.io.gfile.GFile(config_fpath, "r") as f:                                                                                                                                                                                                                     
        proto_str = f.read()                                                                                                                                                                                                                                          
        text_format.Merge(proto_str, pipeline)
    return pipeline, model_ckpt, config_fpath

def modify_config(pipeline, model_ckpt, ft_ckpt_basepath):
    finetune_checkpoint = ft_ckpt_basepath + model_ckpt
    pipeline.model.faster_rcnn.num_classes = num_classes
    pipeline.train_config.fine_tune_checkpoint = finetune_checkpoint
    pipeline.train_config.fine_tune_checkpoint_type = ft_ckpt_type
    pipeline.train_config.batch_size = batch_size
    pipeline.train_config.use_bfloat16 = False # True only if training on TPU

    pipeline.train_input_reader.label_map_path = label_map
    pipeline.train_input_reader.tf_record_input_reader.input_path[0] = train_tfrecord_path

    pipeline.eval_input_reader[0].label_map_path = label_map
    pipeline.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_tfrecord_path

    return pipeline

def write_config(pipeline, config_fpath):
    config_outfpath = os.path.splitext(config_fpath)[0] + '_' + filter + '.config'
    config_text = text_format.MessageToString(pipeline)                                                                                                                                                                                                        
    with tf.io.gfile.GFile(config_outfpath, "wb") as f:                                                                                                                                                                                                                       
        f.write(config_text)
    
    return config_outfpath

def setup_pipeline(model_config, ft_ckpt_basepath):
    print('\n Modifying model config file for {}'.format(model_config))
    pipeline, model_ckpt, config_fpath = read_config(model_config)
    pipeline = modify_config(pipeline, model_ckpt, ft_ckpt_basepath)
    config_outfpath = write_config(pipeline, config_fpath)
    print(' Modifed model config file saved to {}'.format(config_outfpath))
    if config_outfpath:
        return "Success!"
    else:
        return "Fail: try again"

# Modify model configs
model_configs = ['rcnn/pretrained_model/pipeline.config', 'rcnn_i/pretrained_model/pipeline.config']
[setup_pipeline(model_config, ft_ckpt_basepath) for model_config in model_configs]

## Train
--- 

In [ ]:
#@title Determine how many train and eval steps to use based on dataset size

# Get the number of training examples
try: 
    train_image_dir
except:
    train_image_dir = "/content/drive/MyDrive/train/tf2/pre-processing/images" #@param ["/content/drive/MyDrive/train/tf2/images"] {allow-input: true}
examples = len(os.listdir(train_image_dir))
print("Number of train examples: \n", examples)

# Get the number of testing examples
try: 
    test_image_dir
except:
    test_image_dir = "/content/drive/MyDrive/train/tf2/pre-processing/test_images" #@param ["/content/drive/MyDrive/train/tf2/test_images"] {allow-input: true}
test_examples = len(os.listdir(test_image_dir))
print("Number of test examples: \n", test_examples)

# Get the training batch size
try: 
    batch_size
except: 
    batch_size = 1 #@param ["1", "4", "8", "16", "32", "64", "128"] {type:"raw"}
print("Batch size: \n", batch_size)

# Calculate number of steps to use for training and testing based on dataset size
steps_per_epoch = examples / batch_size
num_eval_steps = test_examples / batch_size
print("Number of steps per training epoch: \n", int(steps_per_epoch))
print("Number of evaluation steps: \n", int(num_eval_steps))

In [ ]:
#@title Set training parameters (choose if you want to use RCNN or RCNN_I in form fields on right)

# Choose how many epochs to train for
model_type = "rcnn" #@param ["rcnn", "rcnn_i"] {allow-input: true}
epochs = 10 #@param {type:"slider", min:10, max:1000, step:100}
num_train_steps = int(epochs * steps_per_epoch)
num_eval_steps = int(num_eval_steps)
# Choose paths for RCNN or RCNN_I model
pipeline_config_fn = "pipeline_Multitaxa.config" #@param ["pipeline_Multitaxa.config"] {allow-input: true}
pipeline_config_path = "tf_models/train_demo/" + model_type + "/pretrained_model/" + pipeline_config_fn
model_dir = "tf_models/train_demo/" + model_type + "/trained" 
output_dir = "tf_models/train_demo/" + model_type + "/finetuned_model" 
# Save vars to environment for access with cmd line tools below
os.environ["num_train_steps"] = "num_train_steps"
os.environ["num_eval_steps"] = "num_eval_steps"
os.environ["pipeline_config_path"] = "pipeline_config_path"
os.environ["model_dir"] = "model_dir"
os.environ["output_directory"] = "output_dir"

# Optional: Visualize training progress with Tensorboard?
visualize_with_tensorboard = False #@param {type:"boolean"}
if visualize_with_tensorboard:
    # Load the TensorBoard notebook extension
    %load_ext tensorboard
    # Log training progress using TensorBoard
    %tensorboard --logdir $model_dir

In [ ]:
#@title Train the model
# Note: You can change the number of epochs in code block above, then re-run to train longer
# Modified from https://github.com/RomRoc/objdet_train_tensorflow_colab/blob/master/objdet_custom_tf_colab.ipynb
matplotlib.use('Agg')
%cd $cwd

!python tf_models/models/research/object_detection/model_main_tf2.py \
    --alsologtostderr \
    --num_train_steps=$num_train_steps \
    --num_eval_steps=$num_eval_steps \
    --pipeline_config_path=$pipeline_config_path \
    --model_dir=$model_dir 

In [ ]:
#@title Export trained model
%cd $cwd

# Save the model
!python tf_models/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path=$pipeline_config_path \
    --trained_checkpoint_dir=$model_dir \
    --output_directory=$output_dir

In [ ]:
#@title Evaluate trained model to get mAP and IoU stats for COCO 2017
matplotlib.use('Agg')

!python tf_models/models/research/object_detection/model_main_tf2.py \
    --alsologtostderr \
    --model_dir=$model_dir \
    --pipeline_config_path=$pipeline_config_path \
    --checkpoint_dir=$model_dir